In [33]:
import re

def process_file(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    processed_lines = []
    Z = None  # Initialize Z value
    for line in lines:
        # Match lines containing Z value
        z_match = re.search(r'Z=(\d+\.\d+)', line)
        if z_match:
            Z = float(z_match.group(1))
            processed_lines.append(line)  # Keep the Z line unchanged
            continue

        # Add the new column name if the header line is detected
        if '&X_i,solar' in line:
            header_line = line.strip() + '  &X_i,0\n'
            processed_lines.append(header_line)
            continue

        # Process rows with data
        if Z is not None and re.search(r'&\w+', line):
            columns = line.split('&')
            if len(columns) > 8:  # Ensure it has the X_i,solar column
                try:
                    X_i_solar = float(columns[4].strip())
                    X_i_0 = X_i_solar * Z / 0.0134
                    print(Z)
                    columns.append(f'{X_i_0:.6E}')
                    processed_lines.append('&'.join(columns) + '\n')
                except ValueError:
                    processed_lines.append(line)
            else:
                processed_lines.append(line)
        else:
            processed_lines.append(line)

    with open(output_file, 'w') as file:
        file.writelines(processed_lines)

# Input and output file paths
input_file = '/Users/adriana_work/Desktop/galIMF/yield_tables/TABLE1-VW93ML.txt'
output_file = '/Users/adriana_work/Desktop/galIMF/yield_tables/TABLE1-VW93ML_X_i_0_1.txt'

process_file(input_file, output_file)

In [10]:
import re

# File paths
input_file = "/Users/adriana_work/Desktop/galIMF/yield_tables/TABLE1-VW93ML_X_i_0_1.txt"
output_file = "/Users/adriana_work/Desktop/galIMF/yield_tables/TABLE1-VW93ML_total_yields_scaled_with_X_i_0_total_yield.txt"

# Regular expression to match rows starting with '&'
row_pattern = re.compile(r"^&")

# Read the input file
with open(input_file, 'r') as file:
    lines = file.readlines()

# Process the file
updated_lines = []
for line in lines:
    if row_pattern.match(line):
        # Split the line into columns
        columns = line.split()

        # Extract relevant values
        try:
            p_i = float(columns[1])  # Net Yield
            M_ej = float(columns[2])  # MassExp
            X_i_0 = float(columns[3])  # X_i,0 (last column)

            # Calculate Total yield
            total_yield = p_i + M_ej * X_i_0

            # Update the Total yield column (8th column, index 7)
            columns[4] = f"{total_yield:.6e}"
        except (ValueError, IndexError):
            # Skip rows with invalid data
            pass

        # Reconstruct the line
        line = " ".join(columns) + "\n"

    # Append the processed or unprocessed line
    updated_lines.append(line)

# Write the updated lines back to the file
with open(output_file, 'w') as file:
    file.writelines(updated_lines)

print("Total yield column updated successfully.")


Total yield column updated successfully.
